In [2]:

#setup
import tensorflow as tf
from tensorflow import keras

import IPython
import kerastuner as kt

In [3]:
#load data
(img_train, label_train), (img_test, label_test) = keras.datasets.fashion_mnist.load_data()

In [4]:
#normalize pixel value btwn 1 and 0
img_train = img_train.astype('float32')/255.0
img_test = img_test.astype('float32')/255.0

In [5]:
#define the model
#use the builderfunction
def model_builder(hp):
    model = keras.Sequential()
    model.add(keras.layers.Flatten(input_shape = (28,28)))
    
    #choose units from 32 to 512
    hp_units = hp.Int('units', min_value = 32, max_value = 512, step = 32)
    #tune the no of units in the dense layer
    model.add(keras.layers.Dense(10))
    
    hp_learning_rate = hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])
    #tune the learning_rate for optimizer
    model.compile(optimizer = keras.optimizers.Adam(learning_rate = hp_learning_rate),
                loss = keras.losses.SparseCategoricalCrossentropy(from_logits = True),
                metrics = ['accuracy'])
            
    return model

In [6]:
#instatiante the keras tuner
tuner = kt.Hyperband(model_builder,
                    objective = 'val_accuracy',
                    max_epochs = 10,
                    factor = 3,
                    directory = 'my_dir',
                    project_name = 'intro_to_kt')
class ClearTrainingOutput(tf.keras.callbacks.Callback):
    def on_train_end(*args, **kwargs):
        IPython.display.clear_output(wait = True)

INFO:tensorflow:Reloading Oracle from existing project my_dir/intro_to_kt/oracle.json


JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [ ]:
tuner.search(img_train, label_train, epochs = 10, validation_data = (img_test, label_test), callbacks = [ClearTrainingOutput()])

# Get the optimal hyperparameters
best_hps = tuner.get_best_hyperparameters(num_trials = 1)[0]

print(f"""
The hyperparameter search is complete. The optimal number of units in the first densely-connected
layer is {best_hps.get('units')} and the optimal learning rate for the optimizer
is {best_hps.get('learning_rate')}.
""")

In [ ]:
#retain the model
model = tuner.hypermodel.build(best_hps)
model.fit(img_train, label_train, epochs = 10, validation_data = (img_test, label_test))